In [ ]:
# Optional: mount Google Drive to persist data between sessions
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# Install quran-segmenter (editable) and run the one-shot Colab setup.
!pip install -q -e .

WORDS="/content/qpc-hafs-word-by-word.json"
METADATA="/content/quran-metadata-misc.json"
CONFIG="/content/quran_data/config.json"

!quran-segmenter setup-colab \
  --words "$WORDS" \
  --metadata "$METADATA" \
  --config "$CONFIG"


In [ ]:
# Register your translation.
TRANSLATION_ID="ghali-apapur-apaghuna-clean"
TRANSLATION_FILE="/content/ghali-apapur-apaghuna-clean.json"
TRANSLATION_NAME="Oromo Translation"
LANGUAGE_CODE="om"

!quran-segmenter register "$TRANSLATION_ID" "$TRANSLATION_FILE" \
  --name "$TRANSLATION_NAME" \
  --language "$LANGUAGE_CODE" \
  --spans-embeddings-filepath /content/quran_data/embeddings/spans.npz \
  --segment-embeddings-filepath /content/quran_data/embeddings/${TRANSLATION_ID}.npz


In [ ]:
# Prepare translation (requires GEMINI_API_KEY for segmentation unless you skip it).
# %env GEMINI_API_KEY=your_key
!quran-segmenter prepare "$TRANSLATION_ID"


In [ ]:
# Process audio
AUDIO_FILE="19.mp3"
VERSE_SPEC="19"  # e.g., "taawwudh+basmalah+19" or "2:1-5"

!quran-segmenter process \
  "$AUDIO_FILE" \
  "$VERSE_SPEC" \
  "$TRANSLATION_ID" \
  -o output.json \
  --start-server
